# **Denoising Diffusion Probabilistic Models with MiniAi**

Our custom training library should now allow us to begin building Stable Diffusion from scratch.

We'll be working with the seminal 2020 paper [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) (DDPM) which greatly simplified the training and generation process for these models which were [originally invented in 2015](https://arxiv.org/abs/1503.03585).

In [3]:
import pickle,gzip,math,os,time,shutil,random,logging
import fastcore.all as fc
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter,itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager

from fastcore.foundation import L
import torch
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch import tensor,nn,optim
from torch.utils.data import DataLoader,default_collate
from torch.nn import init
from torch.optim import lr_scheduler
from torcheval.metrics import MulticlassAccuracy
from datasets import load_dataset,load_dataset_builder

from miniai.datasets import *
from miniai.conv import *
from miniai.learner import *
from miniai.activations import *
from miniai.init import *
from miniai.sgd import *
from miniai.resnet import *
from miniai.augment import *

In [5]:
mpl.rcParams['image.cmap'] = 'gray'
logging.disable(logging.WARNING)

## **Load Dataset**

In [6]:
x, y = 'image', 'label'
name = "fashion_mnist"
dsd = load_dataset(name)

We will resize the `28x28` images to `32x32` to make life easier with regard to model architecures.

In [7]:
@inplace
def transformi(b): b[x] = [TF.resize(TF.to_tensor(o), (32, 32)) for o in b[x]]